In [1]:
!pip install -q tensorflow-recommenders
!pip install -q plotnine

In [2]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [3]:
from datetime import datetime
import joblib

import datetime

from typing import List, Union, Dict, Text

import numpy as np
import pandas as pd 
import tensorflow as tf 
import tensorflow_recommenders as tfrs 

import plotnine
import gdown 

2025-02-20 00:00:22.116778: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
data_location = 's3://w210recsys/processed/books_data_clean.pkl'

In [5]:
df = pd.read_pickle(data_location)

In [6]:
df.head()

,id,user_id,title,standard_title,authors,primary_author,categories,parsed_categories,category_cluster,publish_year,review_helpfulness,review_score,review_time,review_summary,review_text,description,embedding_idx
0,0826414346,A30TK6U7DNS82R,Dr. Seuss: American Icon,dr seuss american icon,['Philip Nel'],Philip Nel,['Biography & Autobiography'],[Biography & Autobiography],"Cluster 5: ['History', 'Biography & Autobiogra...",2005,10,5.0,2004-09-21,Really Enjoyed It,I don't care much for Dr. Seuss but after read...,Philip Nel takes a fascinating look into the k...,0
2,0826414346,A2MVUWT453QH61,Dr. Seuss: American Icon,dr seuss american icon,['Philip Nel'],Philip Nel,['Biography & Autobiography'],[Biography & Autobiography],"Cluster 5: ['History', 'Biography & Autobiogra...",2005,7,4.0,2004-07-25,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D...",Philip Nel takes a fascinating look into the k...,1
4,0826414346,A2F6NONFUDB6UK,Dr. Seuss: American Icon,dr seuss american icon,['Philip Nel'],Philip Nel,['Biography & Autobiography'],[Biography & Autobiography],"Cluster 5: ['History', 'Biography & Autobiogra...",2005,2,4.0,2005-09-20,One of America's greatest creative talents,"""Dr. Seuss: American Icon"" by Philip Nel is a ...",Philip Nel takes a fascinating look into the k...,2
5,0826414346,A14OJS0VWMOSWO,Dr. Seuss: American Icon,dr seuss american icon,['Philip Nel'],Philip Nel,['Biography & Autobiography'],[Biography & Autobiography],"Cluster 5: ['History', 'Biography & Autobiogra...",2005,3,5.0,2004-11-11,A memorably excellent survey of Dr. Seuss' man...,Theodor Seuss Giesel was best known as 'Dr. Se...,Philip Nel takes a fascinating look into the k...,3
10,0829814000,A373VVEU6Z9M0N,Wonderful Worship in Smaller Churches,wonderful worship in smaller churches,['David R. Ray'],David R. Ray,['Religion'],[Religion],"Cluster 16: ['Religion', 'Christian life', 'Ch...",2000,1,5.0,2010-12-08,Small Churches CAN Have Wonderful Worship,Many small churches feel like they can not hav...,This resource includes twelve principles in un...,4


In [7]:
df.shape

(394985, 17)

In [8]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [9]:
#### RAM Killer ####
# Define file paths (on Google Drive or local path)
# train_file_path = "/home/sagemaker-user/train_df.csv"
# test_file_path = "/home/sagemaker-user/test_df.csv"

train_file_path = "s3://w210recsys/book_raw/train_df.csv"
test_file_path = "s3://w210recsys/book_raw/test_df.csv"

# Check if the train/test files exist
if not os.path.exists(train_file_path) or not os.path.exists(test_file_path):
    print("Train/test split files do not exist. Creating them now...")

    # Sort the dataframe by user_id and timestamp
    ratings_df = df.sort_values(by=['user_id', 'review_time'])

    # Create train/test splits using groupby and apply with progress bar
    train_df = df.groupby('user_id').progress_apply(lambda x: x.iloc[:-1]).reset_index(drop=True)
    test_df = df.groupby('user_id').progress_apply(lambda x: x.iloc[-1:]).reset_index(drop=True)

    # Save the splits to CSV files on Google Drive
    train_df.to_csv(train_file_path, index=False)
    test_df.to_csv(test_file_path, index=False)

    print(f"Training set saved at: {train_file_path}")
    print(f"Test set saved at: {test_file_path}")
else:
    print(f"Train/test split files already exist. Loading them...")

    # Load the saved train/test splits from CSV files
    train_df = pd.read_csv(train_file_path)
    test_df = pd.read_csv(test_file_path)

# Check the sizes of the datasets
print(f"Training set: {train_df.shape}")
print(f"Test set: {test_df.shape}")

Train/test split files do not exist. Creating them now...


  0%|          | 0/14402 [00:00<?, ?it/s]

  0%|          | 0/14402 [00:00<?, ?it/s]

Training set saved at: s3://w210recsys/book_raw/train_df.csv
Test set saved at: s3://w210recsys/book_raw/test_df.csv
Training set: (380583, 17)
Test set: (14402, 17)


In [10]:
# Convert datasets into tensor datasets
train_ds = tf.data.Dataset.from_tensor_slices(dict(train_df[['user_id', 'title', 'review_score']]))

for x in train_ds.take(5).as_numpy_iterator():
    print(x)

print('\n')

test_ds = tf.data.Dataset.from_tensor_slices(dict(test_df[['user_id', 'title', 'review_score']]))

for x in test_ds.take(5).as_numpy_iterator():
    print(x)

{'user_id': b'A01038432MVI9JXYTTK5T', 'title': b'Journey to the Centre of the Earth', 'review_score': 5.0}
{'user_id': b'A01038432MVI9JXYTTK5T', 'title': b'Twenty thousand leagues under the sea', 'review_score': 5.0}
{'user_id': b'A01038432MVI9JXYTTK5T', 'title': b'20, 000 Leagues Under the Sea', 'review_score': 5.0}
{'user_id': b'A01038432MVI9JXYTTK5T', 'title': b'Twenty Thousand Leagues Under the Sea (Thorndike Press Large Print Perennial Bestsellers Series)', 'review_score': 5.0}
{'user_id': b'A01038432MVI9JXYTTK5T', 'title': b'Journey to the Center of the Earth', 'review_score': 5.0}


{'user_id': b'A01038432MVI9JXYTTK5T', 'title': b'The Mysterious Island (Classics Illustrated)', 'review_score': 5.0}
{'user_id': b'A100NGGXRQF0AQ', 'title': b'Repressed Memories: A Journey to Recovery from Sexual Abuse (Fireside Parkside Books)', 'review_score': 2.0}
{'user_id': b'A100V1W0C8BWOL', 'title': b'The Scarlet Letter, A Romance', 'review_score': 4.0}
{'user_id': b'A100YHBWL4TR4D', 'title': 

2025-02-20 00:01:34.416098: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-02-20 00:01:34.429405: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [11]:
# Create Feature Vocabularies
unique_user_ids = train_df['user_id'].unique()
unique_titles = train_df['title'].unique()
unique_review_scores = train_df['review_score'].unique()

# Candidates for retrieval Task
candidate_ds = tf.data.Dataset.from_tensor_slices(dict(
    train_df[['title']].drop_duplicates()
))

for x in candidate_ds.take(5).as_numpy_iterator():
    print(x)

{'title': b'Journey to the Centre of the Earth'}
{'title': b'Twenty thousand leagues under the sea'}
{'title': b'20, 000 Leagues Under the Sea'}
{'title': b'Twenty Thousand Leagues Under the Sea (Thorndike Press Large Print Perennial Bestsellers Series)'}
{'title': b'Journey to the Center of the Earth'}


In [12]:
len(unique_titles)

70874

In [13]:
# Cache train dataset & Candidate dataset
train_size = train_df.shape[0]
cached_train = train_ds.shuffle(train_size).batch(128).cache()

In [14]:
# User/Query Model
class UserModel(tf.keras.Model):
    '''
    The user(query) tower
    '''

    def __init__(self,
                 unique_user_ids: np.ndarray,
                 feature_user_id_name: str,
                 embedding_dimensions: int):
        '''
        Params
        :param unique_user_ids: array of unique user ids
        :param feature_user_id_name: name of the feature
        :param embedding_dimension: number of dimensions in embedding layer
        '''
        super().__init__()
        self.feature_user_id_name = feature_user_id_name

        self.user_embedding_layers = tf.keras.Sequential(
            [
                tf.keras.layers.StringLookup(
                    vocabulary=unique_user_ids,
                    mask_token=None,
                    name='user_id_vocab',
                ),
                tf.keras.layers.Embedding(
                    input_dim=len(unique_user_ids) + 1,
                    output_dim=embedding_dimensions,
                    name='user_id_embedding',
                ),
            ],
        )

    def call(self, inputs: Dict[Text, tf.Tensor]) -> tf.Tensor:
        return self.user_embedding_layers(inputs[self.feature_user_id_name])

In [15]:
class BookModel(tf.keras.Model):
    '''
    The book(query) tower
    '''

    def __init__(self,
                 unique_titles: np.ndarray,
                 feature_book_title_name: str,
                 embedding_dimensions: int,
                 text_vectorization_max_tokens: int):
        '''
        Params
        :param unique_titles: array of unique titles
        :param unique_review_scores: array of unique review scores
        :param feature_book_title_name: name of the column title
        :param embedding_dimensions: number of dimensions in embedding layer
        :param text_vectorization_max_tokens: maximum number of tokens to vector
        '''
        super().__init__()
        self.feature_book_title_name = feature_book_title_name

        # Book Title embedding
        self.book_embedding_layers = tf.keras.Sequential(
            [
                tf.keras.layers.StringLookup(
                    vocabulary=unique_titles,
                    mask_token=None,
                    name='book_id_vocab',
                ),
                tf.keras.layers.Embedding(
                    input_dim=len(unique_titles) + 1,
                    output_dim=embedding_dimensions,
                    name='book_id_embedding',
                ),
            ],
            name='book_id_embedding',
        )

    def call(self, inputs: Dict[Text, tf.Tensor]) -> tf.Tensor:
        return tf.concat([
            self.book_embedding_layers(inputs[self.feature_book_title_name]),
            # add more embedding layers as needed
        ], axis=1)

In [16]:
class BooksTwoTowersModel(tfrs.Model):
    '''
    Two-Towers books recommender model
    '''
    def __init__(self,
                 unique_user_ids: np.ndarray,
                 unique_titles: np.ndarray,
                 unique_review_scores: np.ndarray,
                 candidate_ds: tf.data.Dataset,
                 feature_user_id_name: str = 'user_id',
                 feature_book_title_name: str = 'title',
                 feature_review_score_name: str = 'review_score',
                 embedding_dimensions: int = 64):
        '''
        Instantiate query tower, candidate tower, and retrieval task.
        '''
        super().__init__()
        self.feature_user_id_name = feature_user_id_name
        self.feature_book_title_name = feature_book_title_name
        self.feature_review_score_name = feature_review_score_name

        # Query Tower
        self.user_model = UserModel(
            unique_user_ids=unique_user_ids,
            feature_user_id_name=feature_user_id_name,
            embedding_dimensions=embedding_dimensions,
        )

        # Candidate Tower
        text_vectorization_max_tokens = len(unique_titles) + len(unique_review_scores)

        book_model_raw = BookModel(
            unique_titles=unique_titles,
            feature_book_title_name=feature_book_title_name,
            embedding_dimensions=embedding_dimensions,
            text_vectorization_max_tokens=text_vectorization_max_tokens,
        )

        # Dense projection layer to equate final tower output dims
        self.book_model = tf.keras.Sequential(
            [
                book_model_raw,
                tf.keras.layers.Dense(
                    units=embedding_dimensions,
                    name='book_dense_projection',
                ),
            ],
            name='book_sequential',
        )

        # Retrieval Task
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=candidate_ds.batch(128).map(self.book_model),
                ks=(10, 20, 50)
            )
        )

    def compute_loss(self,
                     features: Dict[Text, tf.Tensor],
                     training=False) -> tf.Tensor:
        '''
        Get embeddings for users and books.
        Compute dot product and retrieve candidates.
        '''
        user_embeddings = self.user_model({
            self.feature_user_id_name: features[self.feature_user_id_name],
        })

        book_embeddings = self.book_model({
            self.feature_book_title_name: features[self.feature_book_title_name],
        })

        # Sample weight logic
        review_scores = tf.cast(features[self.feature_review_score_name], tf.float32)
        sample_weight = tf.where(review_scores >= 4, 1.0, 0.0)

        return self.task(user_embeddings, book_embeddings, compute_metrics=not training)

In [17]:
# Setup log dir for tensorboard
LOG_DIR = "/home/sagemaker-user/logs"

if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR)

In [18]:
# Compile Model
model = BooksTwoTowersModel(
    unique_user_ids=unique_user_ids,
    unique_titles=unique_titles,
    unique_review_scores=unique_review_scores,
    candidate_ds=candidate_ds,
    embedding_dimensions=64,
)

model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [19]:
# Train
model.fit(
    cached_train,
    epochs=10,
    callbacks=[tensorboard_callback],
)

Epoch 1/10
2974/2974 [==============================] - 8s 2ms/step - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_20_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - loss: 555.9264 - regularization_loss: 0.0000e+00 - total_loss: 555.9264
Epoch 2/10
2974/2974 [==============================] - 6s 2ms/step - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_20_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - loss: 410.4477 - regularization_loss: 0.0000e+00 - total_loss: 410.4477
Epoch 3/10
2974/2974 [==============================] - 6s 2ms/step - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_20_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - loss: 293.7381 - regularization_loss: 0.0000e+00 - total_loss: 293.7381
Epoch 4/10
2974/2974 [=====================

In [ ]:
book_embeddings = model.book_model.layers[0].book_embedding_layers.get_weights()[0]

In [ ]:
user_embeddings = model.user_model.user_embedding_layers.get_weights()[0]
user_embeddings.shape

In [ ]:
df.head(2)

In [ ]:
test_user = tf.data.Dataset.from_tensor_slices({
    'user_id': test_df['user_id'].sample(1).values,
    'title': test_df['title'].sample(1).values,
    'review_score': test_df['review_score'].sample(1).values,
})

In [ ]:
user_embeddings = model.user_model.predict(test_user)

In [ ]:
user_embeddings.shape == book_embeddings[1].shape

In [ ]:
book_embeddings.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
user_embeddings_2d = np.expand_dims(user_embeddings, axis=0)
cosine_sim = cosine_similarity(user_embeddings_2d, book_embeddings)
cosine_sim

In [ ]:
len(cosine_sim[0])

In [ ]:
top_n = 10
top_n_indicies = np.argsort(cosine_sim[0])[::-1][:top_n]

In [ ]:
top_n_indicies

In [ ]:
for i in top_n_indicies:
    print(unique_titles[i-1])
    print("")

In [ ]:
# Fasten Your Seat Belts! History and Heroism in the Pan Am Cabin

# Twice Loved

# Agent of Vega

# Pagan Heat

# Freedom's Price (Loveswept)

# Looneyspoons: Low-Fat Food Made Fun!

# The Bluebird and the Sparrow (Women of the West #10) (Janette Oke Classics for Girls)

# How to Start a Business in New York City

# Transmitter Hunting: Radio Direction Finding Simplified

# The Staircase

In [ ]:
row=unique_titles[6892]
row

In [ ]:
# '''
# Params
# :param unique_titles: array of unique titles
# :param unique_review_scores: array of unique review scores
# :param feature_book_title_name: name of the column title
# :param embedding_dimensions: number of dimensions in embedding layer
# :param text_vectorization_max_tokens: maximum number of tokens to vector
# '''
t = []
r = []
t.append(df.title.iloc[1])
r.append(df.review_score.iloc[1])
# r = list(df.review_score.iloc[1])
c = 'title'
d = 64
v = 100

In [ ]:
model = SageModel(
    model=inference.py
)
